In [1]:


#importing the required packages

import cv2
from skimage import data, io, filters
from PIL import Image
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler



In [2]:

#Reading the train and test data

train = pd.read_csv('D:\Kaggle Projects\Leaf Classification\\train\\train.csv')
test_1 = pd.read_csv('D:\Kaggle Projects\Leaf Classification\\test\\test.csv')


In [3]:
# Taking the Id columns from train and Test as well as species from train data 
train_head = train.iloc[:,0]
test_head = test_1.iloc[:,0]
train_species = train.iloc[:,1]

In [4]:

le = preprocessing.LabelEncoder()
y_species =  le.fit_transform(train.species)


In [5]:

#Initializing the List of the different values 
lst_area = []
lst_perimeter = []
ratio = []
extent_1 = []
diameter = []


# Initializing the path of the Different images 

path = "D:\\Kaggle Projects\\Leaf Classification\\images\\"

#Generating the values of Area , perimeter, diameter using the loop passing id's in the train data
for values in train_head:
    #Getting the values from the train header file 
    num = values
    #Converting the values in the string so that it can be further be added to another string 
    num = str(num)
    # Adding the Extention to the file path 
    val = num + '.jpg'
    #Taking the full path values in the val2 
    val2 = path+val
    # Reading the Image from the file 
    img = cv2.imread(val2,0)
    # Extracting the Different features from the Images 
    ret,thresh = cv2.threshold(img,127,255,0)
    # Getting the contour from the image 
    contours,hierarchy = cv2.findContours(thresh,1,2)
    cnt = contours[0]
    M = cv2.moments(cnt)
    # Getting the Contour ARea from the Image 
    area = cv2.contourArea(cnt)
    area = round(area, 2)
    # Appending diffrent ares in the image 
    lst_area.append(area)
    # Same caclulation for perimeter , aspect ratio and diameter and further add them in the list
    perimeter = cv2.arcLength(cnt,True)
    perimeter = round(perimeter, 2)
    lst_perimeter.append(perimeter)
    x,y,w,h = cv2.boundingRect(cnt)
    aspect_ratio = float(w)/h
    ratio.append(aspect_ratio)
    x,y,w,h = cv2.boundingRect(cnt)
    rect_area = w*h
    extent = float(area)/rect_area
    extent = round(extent, 2)
    extent_1.append(extent)
    equi_diameter = np.sqrt(4*area/np.pi)
    diameter.append(equi_diameter)
   



In [6]:
# Adding the Perimeter , Area , Ratio , Extent and Diameter Diffrent values from list to data frame and renaming the column in the
# Data Frame
Perimeter_train  = pd.DataFrame(lst_perimeter)
Perimeter_train.columns = ['Perimeter']
Area_train  = pd.DataFrame(lst_area)
Area_train.columns = ['Area']
Ratio_train  = pd.DataFrame(ratio)
Ratio_train.columns = ['Ratio']
Extent_train  = pd.DataFrame(extent_1)
Extent_train.columns = ['Extent']
Dia_train  = pd.DataFrame(diameter)
Dia_train.columns = ['Diameter']


In [7]:
# Same as above Train Data

lst_area = []
lst_perimeter = []
ratio = []
extent_1 = []
diameter = []
size = []

path = "D:\\Kaggle Projects\\Leaf Classification\\images\\"

#Generating the values of Area , perimeter, diameter using the loop passing id's in the train data
for values in test_head:
    num = values
    num = str(num)
    val = num + '.jpg'
    val2 = path+val
    img = cv2.imread(val2,0)
    ret,thresh = cv2.threshold(img,127,255,0)
    contours,hierarchy = cv2.findContours(thresh, 1, 2)
    cnt = contours[0]
    M = cv2.moments(cnt)
    area = cv2.contourArea(cnt)
    area = round(area, 2)
    lst_area.append(area)
    perimeter = cv2.arcLength(cnt,True)
    perimeter = round(perimeter, 2)
    lst_perimeter.append(perimeter)
    x,y,w,h = cv2.boundingRect(cnt)
    aspect_ratio = float(w)/h
    ratio.append(aspect_ratio)
    x,y,w,h = cv2.boundingRect(cnt)
    rect_area = w*h
    extent = float(area)/rect_area
    extent = round(extent, 2)
    extent_1.append(extent)
    equi_diameter = np.sqrt(4*area/np.pi)
    diameter.append(equi_diameter)


In [8]:
# Adding the Perimeter , Area , Ratio , Extent and Diameter Diffrent values from list to data frame and renaming the column in the
# Data Frame
Perimeter_test  = pd.DataFrame(lst_perimeter)
Perimeter_test.columns = ['Perimeter']
Area_test  = pd.DataFrame(lst_area)
Area_test.columns = ['Area']
Ratio_test  = pd.DataFrame(ratio)
Ratio_test.columns = ['Ratio']
Extent_test  = pd.DataFrame(extent_1)
Extent_test.columns = ['Extent']
Dia_test  = pd.DataFrame(diameter)
Dia_test.columns = ['Diameter']


In [9]:
# Droping the Species Column as we have converted the Species from categorical values to numerical values 
train_new = train.drop('species', 1)



In [10]:

#Adding the below findings from the Images to the Data frame , i.e train file 

train_new['Perimeter'] =  Perimeter_train
train_new['Area'] =  Area_train
train_new['Ratio'] =  Ratio_train
train_new['Extent'] =  Extent_train
train_new['Diameter'] =  Dia_train


In [11]:

#Adding the below findings from the Images to the Data frame , i.e test file 
test_1['Perimeter'] =  Perimeter_test
test_1['Area'] =  Area_test
test_1['Ratio'] =  Ratio_test
test_1['Solidity'] =  Extent_test
test_1['Diameter'] =  Dia_test


In [12]:

# Encoding the Diffrent categorical species to the NUmerical values
# for example Acer_Opalus to 1 , Quercus_Hartwissiana to 2 like wise
label = LabelEncoder().fit(train['species'])
y_train = label.transform(train['species'])



In [13]:
# Converting the Data into the Scaler format 
# Scaler fit means all the variances been ajusted i.e normalized the Data 
# Both train and Test Data be Standarized 
scaler = StandardScaler().fit(train_new)
x_train = scaler.transform(train_new)
scaler = StandardScaler().fit(test_1)
x_test = scaler.transform(test_1)


In [14]:
#Generating the model
# Using Multinomial function as we have different categorical values as well as tuned the parameter by taking the
#C as 1000 and tol as 0.0008 
# In the above two kernals we have used lbfgs for better performance
clf = LogisticRegression(solver='lbfgs' , multi_class='multinomial', C=1000, tol=0.0008)
clf.fit(x_train, y_train)



LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0008, verbose=0, warm_start=False)

In [15]:

# Printing the Model score from both train and test data 
print(clf.score(train_new, y_train))

0.010101010101


In [16]:
#Predicting the values using the trained model
y_test = clf.predict_proba(x_test)


# In[26]:

# Generating the subm,ission file and save it to the specific location
submission = pd.DataFrame(y_test, index=test_head, columns=le.classes_)
submission.to_csv('D:\Kaggle Projects\Leaf Classification\\submission_model.csv')
